In [1]:
import pandas as pd
# 파일을 DataFrame으로 읽어오는 코드 예시
df = pd.read_csv("//home/iai/Desktop/son/capstone/classimbalance/cifar100-lt/imb_fact_100/total/results/experiment_results.txt", sep=", ")

# 데이터 타입 변환
df["ACC Mean"] = df["ACC Mean"].astype(float)
df["ACC Std"] = df["ACC Std"].astype(float)
df["ECE Mean"] = df["ECE Mean"].astype(float)
df["ECE Std"] = df["ECE Std"].astype(float)

# DataFrame 확인
df.head()


/tmp/ipykernel_1658289/1162723274.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv("//home/iai/Desktop/son/capstone/classimbalance/cifar100-lt/imb_fact_100/total/results/experiment_results.txt", sep=", ")


KD Method Teacher Calibration Student Calibration        Mixup  ACC Mean  \
0       VKD               TNone                  CE  Mixup=False     37.88   
1       VKD               TNone                  CE   Mixup=True     38.98   
2       VKD               TNone                 ALS  Mixup=False     38.73   
3       VKD               TNone                 ALS   Mixup=True     33.27   
4       VKD               TNone   ConfidencePenalty  Mixup=False     36.34   

   ACC Std  ECE Mean  ECE Std  
0     0.60    0.1578   0.0053  
1     0.41    0.1453   0.0095  
2     0.32    0.1767   0.0059  
3     0.90    0.1454   0.0048  
4     0.36    0.1813   0.0184

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# 데이터 타입 변환
df["ACC Mean"] = df["ACC Mean"].astype(float)
df["ECE Mean"] = df["ECE Mean"].astype(float)

# 데이터 정규화 및 조화평균 계산
df["Normalized ACC"] = (df["ACC Mean"] - df["ACC Mean"].min()) / (df["ACC Mean"].max() - df["ACC Mean"].min())
df["Normalized ECE"] = (df["ECE Mean"] - df["ECE Mean"].min()) / (df["ECE Mean"].max() - df["ECE Mean"].min())

df["Normalized Harmonic Score"] = 2 * (
    df["Normalized ACC"] * (1 - df["Normalized ECE"])
) / (df["Normalized ACC"] + (1 - df["Normalized ECE"]))

# Student Calibration 약어 변경
df["Student Calibration"] = df["Student Calibration"].replace({
    "BalancedSoftmax": "BS",
    "ConfidencePenalty": "CP"
})

# 순서 지정
calibration_order = ["CE", "ALS", "CP", "BS"]

# 각 기법별 평균 계산
calibration_avg = (
    df.groupby("Student Calibration")[["ACC Mean", "ECE Mean", "Normalized Harmonic Score"]]
    .mean()
    .reindex(calibration_order)
    .reset_index()
)

# 각 기법별 평균 출력
print(calibration_avg.to_string(index=False))

# X축 위치 설정
x = np.arange(len(calibration_avg["Student Calibration"]))
width = 0.2  # 막대의 너비

# 그래프 그리기
fig, ax1 = plt.subplots(figsize=(14, 8))

# 첫 번째 Y축 (ACC Mean)
bar1 = ax1.bar(
    x - width, 
    calibration_avg["ACC Mean"], 
    width, 
    color='royalblue', 
    alpha=0.7, 
    label='ACC Mean'
)
ax1.set_xlabel('Student Calibration', fontsize=14)
ax1.set_ylabel('ACC Mean', fontsize=14, color='royalblue')
ax1.tick_params(axis='y', labelcolor='royalblue')
ax1.set_xticks(x)
ax1.set_xticklabels(calibration_avg["Student Calibration"], rotation=45, ha='right')

# 두 번째 Y축 (ECE Mean)
ax2 = ax1.twinx()
bar2 = ax2.bar(
    x, 
    calibration_avg["ECE Mean"], 
    width, 
    color='lightcoral', 
    alpha=0.7, 
    label='ECE Mean'
)
ax2.set_ylabel('ECE Mean', fontsize=14, color='lightcoral')
ax2.tick_params(axis='y', labelcolor='lightcoral')

# 세 번째 Y축 (Normalized Harmonic Score)
ax3 = ax1.twinx()
ax3.spines['right'].set_position(('outward', 60))  # 세 번째 Y축 위치 조정
bar3 = ax3.bar(
    x + width, 
    calibration_avg["Normalized Harmonic Score"], 
    width, 
    color='green', 
    alpha=0.7, 
    label='Normalized Harmonic Score'
)
ax3.set_ylabel('Normalized Harmonic Score', fontsize=14, color='green')
ax3.tick_params(axis='y', labelcolor='green')

# 범례 추가
bars = [bar1, bar2, bar3]
labels = [bar.get_label() for bar in bars]
fig.legend(bars, labels, loc='upper center', bbox_to_anchor=(0.5, 1.05), ncol=3, fontsize=12)

# 그래프 제목 및 레이아웃
fig.suptitle('Comparison of ACC Mean, ECE Mean, and Normalized Harmonic Score', fontsize=16)
fig.tight_layout()
plt.show()


Student Calibration  ACC Mean  ECE Mean  Normalized Harmonic Score
                 CE 39.246563  0.162042                   0.623848
                ALS 36.618437  0.153273                   0.464500
                 CP 39.214531  0.182859                   0.460482
                 BS 39.110312  0.164783                   0.600388


In [3]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 8))

# 4개의 Calibration 기법 분리
calibration_types = ["CE", "ALS", "CP", "BS"]
colors = ["orange", "purple", "blue", "green"]

# Calibration 기법별로 데이터 표시
for calibration, color in zip(calibration_types, colors):
    calibration_data = df[df["Student Calibration"] == calibration]
    plt.scatter(
        calibration_data["ECE Mean"], 
        calibration_data["ACC Mean"], 
        color=color, 
        alpha=0.6, 
        label=f"Calibration: {calibration}"
    )

# 평균선
ece_mean = df["ECE Mean"].mean()
acc_mean = df["ACC Mean"].mean()
plt.axvline(ece_mean, color="blue", linestyle="--", linewidth=1.5, label="Mean ECE")
plt.axhline(acc_mean, color="green", linestyle="--", linewidth=1.5, label="Mean ACC")

# Ideal Region 표시 (ECE <= Mean ECE AND ACC >= Mean ACC)
plt.fill_betweenx(
    y=[acc_mean, df["ACC Mean"].max()],  # ACC 범위
    x1=0, x2=ece_mean,                  # ECE 범위
    color='green', alpha=0.3, label="Ideal Region"
)

# 그래프 설정
plt.title("Scatter Plot by Student Calibration with Ideal Region", fontsize=14)
plt.xlabel("ECE Mean", fontsize=12)
plt.ylabel("ACC Mean", fontsize=12)
plt.xlim(0.10, df["ECE Mean"].max())
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()

# 그래프 표시
plt.show()


In [4]:
# 필요한 데이터가 로드된 상태에서 실행
df["ACC Mean"] = df["ACC Mean"].astype(float)
df["ECE Mean"] = df["ECE Mean"].astype(float)

# Teacher-Student 조합 이름 생성
df['Teacher-Student Combination'] = df['Teacher Calibration'] + ' -> ' + df['Student Calibration']

# 데이터 정규화
df["Normalized ACC"] = (df["ACC Mean"] - df["ACC Mean"].min()) / (df["ACC Mean"].max() - df["ACC Mean"].min())
df["Normalized ECE"] = (df["ECE Mean"] - df["ECE Mean"].min()) / (df["ECE Mean"].max() - df["ECE Mean"].min())

# 정규화된 조화평균 계산
df["Normalized Harmonic Score"] = 2 * (
    df["Normalized ACC"] * (1 - df["Normalized ECE"])
) / (df["Normalized ACC"] + (1 - df["Normalized ECE"]))

# 상위 5개 및 하위 5개 Normalized Harmonic Score 추출
top_5_harmonic = df.nlargest(5, "Normalized Harmonic Score")
bottom_5_harmonic = df.nsmallest(5, "Normalized Harmonic Score")

# 결과
print(top_5_harmonic[['Teacher-Student Combination', 'ACC Mean', 'ECE Mean', 'Normalized Harmonic Score']])
print(bottom_5_harmonic[['Teacher-Student Combination', 'ACC Mean', 'ECE Mean', 'Normalized Harmonic Score']])


   Teacher-Student Combination  ACC Mean  ECE Mean  Normalized Harmonic Score
83                  TCP -> ALS     41.86    0.1493                   0.799217
79                  TALS -> BS     42.24    0.1522                   0.790367
97              TNone_MX -> CE     42.06    0.1520                   0.786839
81                   TCP -> CE     41.95    0.1522                   0.782427
75                 TALS -> ALS     40.77    0.1489                   0.767691
    Teacher-Student Combination  ACC Mean  ECE Mean  Normalized Harmonic Score
84                    TCP -> CP     41.90    0.2194                   0.000000
155                  TBS -> ALS     30.28    0.1188                   0.000000
187               TBS_MX -> ALS     31.05    0.1288                   0.116534
251               TBS_MX -> ALS     31.32    0.1309                   0.153594
163             TNone_MX -> ALS     31.33    0.1267                   0.155562
